# Recommending a restaurant location in Northern Suburbs of Melbourne

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
!pip install geopy
!pip install pgeocode
import pgeocode
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
! pip install folium==0.5.0
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [2]:
URL = "https://postcodes-australia.com/state-postcodes/vic"
page = requests.get(URL)
soup = BeautifulSoup(page.content,'html.parser')

In [3]:
sublist = []
viclist = soup.find_all(class_ = "pclist")
i=0
for sub in viclist:
    temp = {}
    ulList = sub.find_all('li')
    #print(ulList)
    for li in ulList:
        alist = li.find_all('a')
        for a in alist:
            
            if a == None or a.text == 'Back to top of page' or a.text == 'Return to home page':
                pass
            else:
                x = int(a.text)
                ul = li.find_all('ul')
                for l in ul:
                    if l == None:
                        pass
                    else:
                        temp = {}
                        if ((x > 3000) and (x < 3100)):
                            temp['Place'] = l.text.strip('\n').split('\n')[0]
                            temp['Postcode'] = a.text
                       # print(temp)
                            sublist.append(temp)
df = pd.DataFrame(sublist)
df.shape

(89, 2)

In [4]:
pgeocode.Nominatim('au')
geolocator = pgeocode.Nominatim('au')
postal_codes = df['Postcode'].tolist() #to list
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    g = geolocator.query_postal_code(postal_code)
    if not g.empty:
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

df['Latitude'] = latitudes
df['Longitude'] = longitudes

#drop any Nan values from latitude and longitude
df = df.dropna(subset=['Latitude'])
df = df.dropna(subset=['Longitude'])


df


,Place,Postcode,Latitude,Longitude
0,Melbourne,3001,-37.8140,144.963300
1,East Melbourne,3002,-37.8133,144.983900
2,West Melbourne,3003,-37.8088,144.947300
3,Melbourne,3004,-37.8140,144.967600
4,World Trade Centre,3005,-37.8152,144.963200
...,...,...,...,...
84,Montmorency,3094,-37.7188,145.124500
85,Eltham,3095,-37.7043,145.158267
86,Wattle Glen,3096,-37.6672,145.184900
87,Bend Of Islands,3097,-37.7018,145.257433


In [5]:
address = 'Melbourne'

geolocator = Nominatim(user_agent="au_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [6]:
map_melb = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Place']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melb)  
    
map_melb

In [7]:
CLIENT_ID = 'KKZ1PUOXHJI2MDTUF5NCUPRSET2YYJB54HOVXWSJICKMV3ID' # your Foursquare ID
CLIENT_SECRET = 'YBWRJSWWCS1EZE0RPWE1DL01OR4KKQQWMG5WGEBKHB3PGUF2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [8]:
df.loc[0, 'Place']

'Melbourne'

In [9]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Place'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Melbourne are -37.814, 144.9633.


In [10]:
LIMIT=100
radius=2000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId=4d4b7105d754a06374d81259&categoryId!=4bf58dd8d48988d10f941735&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = requests.get(url).json()['response']['groups'][0]['items']
nearby_venues =[]
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply([get_category_type], axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Tipo 00,Italian Restaurant,-37.813527,144.961978
1,B'cos Brazil,Brazilian Restaurant,-37.815486,144.963085
2,Trattoria Emilia,Italian Restaurant,-37.815220,144.962636
3,Calia,Japanese Restaurant,-37.812724,144.963930
4,Nosh,Asian Restaurant,-37.815396,144.962999
...,...,...,...,...
95,ENA greek street food,Greek Restaurant,-37.819897,144.966001
96,Harley House,BBQ Joint,-37.814278,144.971471
97,Simply Spanish,Spanish Restaurant,-37.816208,144.958411
98,Chocolateria San Churro,Café,-37.810482,144.964394


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId=4d4b7105d754a06374d81259&categoryId!=4bf58dd8d48988d10f941735&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
melb_venues = getNearbyVenues(names=df['Place'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
melb_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne,-37.8140,144.963300,Tipo 00,-37.813527,144.961978,Italian Restaurant
1,Melbourne,-37.8140,144.963300,B'cos Brazil,-37.815486,144.963085,Brazilian Restaurant
2,Melbourne,-37.8140,144.963300,Trattoria Emilia,-37.815220,144.962636,Italian Restaurant
3,Melbourne,-37.8140,144.963300,Calia,-37.812724,144.963930,Japanese Restaurant
4,Melbourne,-37.8140,144.963300,Nosh,-37.815396,144.962999,Asian Restaurant
...,...,...,...,...,...,...,...
948,Eltham,-37.7043,145.158267,House Of Salad,-37.704270,145.160196,Pizza Place
949,Eltham,-37.7043,145.158267,Wild Bean Cafe,-37.704172,145.160553,Café
950,Eltham,-37.7043,145.158267,Wild Bean Cafe,-37.704151,145.160565,Café
951,Eltham,-37.7043,145.158267,Nanna G's,-37.702307,145.155719,Australian Restaurant


In [15]:
melb_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford,12,12,12,12,12,12
Aberfeldie,5,5,5,5,5,5
Airport West,3,3,3,3,3,3
Albanvale,1,1,1,1,1,1
Albion,5,5,5,5,5,5
...,...,...,...,...,...,...
West Melbourne,27,27,27,27,27,27
Williamstown,5,5,5,5,5,5
World Trade Centre,100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(melb_venues['Venue Category'].unique())))

There are 82 uniques categories.


In [17]:
melb_onehot = pd.get_dummies(melb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
melb_onehot['Neighborhood'] = melb_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [melb_onehot.columns[-1]] + list(melb_onehot.columns[:-1])
melb_onehot = melb_onehot[fixed_columns]

melb_onehot

,Neighborhood,Afghan Restaurant,African Restaurant,Argentinian Restaurant,Armenian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,...,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yunnan Restaurant
0,Melbourne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Melbourne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Melbourne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Melbourne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Melbourne,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Eltham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
949,Eltham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
950,Eltham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
951,Eltham,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
melb_grouped = melb_onehot.groupby('Neighborhood').mean().reset_index()
melb_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,Argentinian Restaurant,Armenian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,...,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yunnan Restaurant
0,Abbotsford,0.0,0.0,0.00,0.0,0.083333,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.083333,0.000000,0.0
1,Aberfeldie,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.20,0.0,0.000000,0.000000,0.0
2,Airport West,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0
3,Albanvale,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,1.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0
4,Albion,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,West Melbourne,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.037037,0.0
61,Williamstown,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.20,0.0,0.000000,0.000000,0.0
62,World Trade Centre,0.0,0.0,0.01,0.0,0.020000,0.01,0.0,0.01,0.01,...,0.02,0.0,0.05,0.0,0.02,0.02,0.0,0.010000,0.020000,0.0
63,Yarrambat,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0


In [19]:
num_top_venues = 5

for hood in melb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = melb_grouped[melb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbotsford----
                           venue  freq
0                           Café  0.50
1            Japanese Restaurant  0.08
2  Vegetarian / Vegan Restaurant  0.08
3               Asian Restaurant  0.08
4             Italian Restaurant  0.08


----Aberfeldie----
               venue  freq
0               Café   0.4
1    Thai Restaurant   0.2
2             Bakery   0.2
3         Food Court   0.2
4  Afghan Restaurant   0.0


----Airport West----
                       venue  freq
0         Italian Restaurant  0.33
1          Fish & Chips Shop  0.33
2                     Bakery  0.33
3  Middle Eastern Restaurant  0.00
4           Ramen Restaurant  0.00


----Albanvale----
                       venue  freq
0                  BBQ Joint   1.0
1          Afghan Restaurant   0.0
2  Middle Eastern Restaurant   0.0
3                 Restaurant   0.0
4           Ramen Restaurant   0.0


----Albion----
                venue  freq
0  Chinese Restaurant   0.2
1         Pizza Place   0.2


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = melb_grouped['Neighborhood']

for ind in np.arange(melb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(melb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Italian Restaurant,Asian Restaurant,Japanese Restaurant,Burger Joint,Yunnan Restaurant,Fast Food Restaurant,Donut Shop
1,Aberfeldie,Café,Bakery,Thai Restaurant,Food Court,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
2,Airport West,Bakery,Italian Restaurant,Fish & Chips Shop,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
3,Albanvale,BBQ Joint,Yunnan Restaurant,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
4,Albion,Chinese Restaurant,Gastropub,Pizza Place,Café,Bakery,Yunnan Restaurant,Fast Food Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
60,West Melbourne,Café,Pizza Place,Sandwich Place,Italian Restaurant,Food Truck,Bakery,Noodle House,Mexican Restaurant,Bistro,Himalayan Restaurant
61,Williamstown,Food,Fast Food Restaurant,Pizza Place,Thai Restaurant,Italian Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
62,World Trade Centre,Café,Italian Restaurant,Korean Restaurant,Mexican Restaurant,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Dumpling Restaurant,Burger Joint,Bakery
63,Yarrambat,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant


In [22]:
kclusters = 4

melb_grouped_clustering = melb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(melb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 2, 1, 0, 1, 1, 2, 3], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

melb_merged = df

In [24]:
melb_merged = melb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Place')

In [25]:
melb_merged=melb_merged.dropna(axis=0,inplace=False)

melb_merged

,Place,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,3001,-37.8140,144.963300,1.0,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
1,East Melbourne,3002,-37.8133,144.983900,1.0,Café,Australian Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Indian Restaurant,Italian Restaurant,Fish & Chips Shop,Falafel Restaurant,Diner
2,West Melbourne,3003,-37.8088,144.947300,1.0,Café,Pizza Place,Sandwich Place,Italian Restaurant,Food Truck,Bakery,Noodle House,Mexican Restaurant,Bistro,Himalayan Restaurant
3,Melbourne,3004,-37.8140,144.967600,1.0,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
4,World Trade Centre,3005,-37.8152,144.963200,1.0,Café,Italian Restaurant,Korean Restaurant,Mexican Restaurant,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Dumpling Restaurant,Burger Joint,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Diamond Creek,3089,-37.6728,145.153700,1.0,Café,Fish & Chips Shop,Malay Restaurant,Fast Food Restaurant,Yunnan Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
82,Yarrambat,3091,-37.6390,145.133400,0.0,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
84,Montmorency,3094,-37.7188,145.124500,1.0,Café,Chinese Restaurant,Thai Restaurant,Burger Joint,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
85,Eltham,3095,-37.7043,145.158267,1.0,Café,Pizza Place,Australian Restaurant,Yunnan Restaurant,Fish & Chips Shop,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(melb_merged['Latitude'], melb_merged['Longitude'], melb_merged['Place'], melb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
melb_merged.loc[melb_merged['Cluster Labels'] == 0, melb_merged.columns[[1] + list(range(5, melb_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,3010,Café,Japanese Restaurant,Pizza Place,Chinese Restaurant,Indian Restaurant,Food Truck,Sushi Restaurant,Food Court,Middle Eastern Restaurant,Mexican Restaurant
38,3044,Café,Snack Place,Yunnan Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
44,3050,Café,Thai Restaurant,Japanese Restaurant,Food Court,Mexican Restaurant,Bistro,Malay Restaurant,Yunnan Restaurant,Fast Food Restaurant,Doner Restaurant
45,3051,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
62,3068,Café,Pizza Place,Yunnan Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
65,3072,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
70,3078,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
71,3079,Café,Restaurant,Yunnan Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
74,3083,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
82,3091,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant


In [28]:
melb_merged.loc[melb_merged['Cluster Labels'] == 1, melb_merged.columns[[1] + list(range(5, melb_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3001,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
1,3002,Café,Australian Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Indian Restaurant,Italian Restaurant,Fish & Chips Shop,Falafel Restaurant,Diner
2,3003,Café,Pizza Place,Sandwich Place,Italian Restaurant,Food Truck,Bakery,Noodle House,Mexican Restaurant,Bistro,Himalayan Restaurant
3,3004,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
4,3005,Café,Italian Restaurant,Korean Restaurant,Mexican Restaurant,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Dumpling Restaurant,Burger Joint,Bakery
5,3006,Café,Asian Restaurant,Restaurant,Burger Joint,Fast Food Restaurant,Sandwich Place,Caucasian Restaurant,Mexican Restaurant,Paella Restaurant,Pizza Place
6,3008,Café,Restaurant,Italian Restaurant,Chinese Restaurant,Pizza Place,Seafood Restaurant,Sandwich Place,Middle Eastern Restaurant,Steakhouse,Australian Restaurant
8,3011,Vietnamese Restaurant,Café,Asian Restaurant,Pizza Place,Malay Restaurant,Portuguese Restaurant,Restaurant,Chinese Restaurant,Burger Joint,Fast Food Restaurant
9,3012,Food Truck,Breakfast Spot,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
10,3013,Fast Food Restaurant,Fried Chicken Joint,Food Truck,Malay Restaurant,Yunnan Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant


In [29]:
melb_merged.loc[melb_merged['Cluster Labels'] == 2, melb_merged.columns[[1] + list(range(5, melb_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,3019,Bakery,Fast Food Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,German Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
16,3021,BBQ Joint,Yunnan Restaurant,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
18,3023,Fast Food Restaurant,Bakery,Noodle House,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
20,3025,Fast Food Restaurant,Sandwich Place,Yunnan Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
39,3045,Bakery,Café,Fast Food Restaurant,Food,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fish & Chips Shop
46,3052,Bakery,Fast Food Restaurant,BBQ Joint,Food,Food Court,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
68,3075,Bakery,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop


In [30]:
melb_merged.loc[melb_merged['Cluster Labels'] == 3, melb_merged.columns[[1] + list(range(5, melb_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,3031,Pizza Place,Fast Food Restaurant,Café,Yunnan Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
27,3032,Pizza Place,Café,Middle Eastern Restaurant,Yunnan Restaurant,Fast Food Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
29,3034,Pizza Place,Bakery,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
37,3043,Fish & Chips Shop,Pizza Place,Deli / Bodega,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
42,3048,Middle Eastern Restaurant,Bakery,Pizza Place,Fish & Chips Shop,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
77,3086,Pizza Place,Café,Yunnan Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
78,3087,Gastropub,Pizza Place,Yunnan Restaurant,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant


In [31]:
melb_merged.loc[melb_merged['Cluster Labels'] == 4, melb_merged.columns[[1] + list(range(5, melb_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [32]:
pip install tabula-py

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 11.7 MB 12.8 MB/s eta 0:00:01    |█▋                              | 583 kB 12.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import tabula
 
file = "https://www.multicultural.vic.gov.au/images/2016_Census/CommunityProfiles2016/India-Community-Profile-2016-Census.pdf"
 
tabula.convert_into(file, "iris_all.csv", pages = 10)
poplist = pd.read_csv("iris_all.csv")

Got stderr: Apr 08, 2021 12:59:14 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Apr 08, 2021 12:59:14 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 08, 2021 12:59:14 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 08, 2021 12:59:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 08, 2021 12:59:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 08, 2021 12:59:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 08, 2021 12:59:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [34]:
popdf = pd.DataFrame(poplist)
popdf.drop("Rank.1", axis=1, inplace=True)
popdf.drop("Unnamed: 2", axis=1, inplace=True)
popdf

,Rank,Unnamed: 3
0,in 2016,"Suburbs with most India-born, ranked"
1,1,"Tarneit, Point Cook, Truganina, Hoppers Crossi..."
2,2,"Hampton Park, Cranbourne East, Berwick, Narre ..."
3,3,"Noble Park, Dandenong, Springvale (Vic.), Keys..."
4,4,"Epping (Vic.), Wollert, South Morang, Lalor, T..."
5,5,"Glen Waverley, Clayton, Mount Waverley, Mulgra..."
6,6,"St Albans (Vic.), Sydenham (Vic.), Deer Park, ..."
7,7,"Craigieburn, Roxburgh Park, Broadmeadows (Vic...."
8,8,"Carnegie, Glen Huntly, Bentleigh East, Ormond,..."
9,9,"Melbourne, Docklands, Southbank, Carlton (Vic...."


In [35]:
poplist = (popdf['Unnamed: 3'].str.replace(" ","")).str.split(",")
poplist[2]

['HamptonPark',
 'CranbourneEast',
 'Berwick',
 'NarreWarrenSouth',
 'CranbourneNorth']

In [36]:
dfpop = pd.DataFrame(poplist[1])
dfpop = dfpop.append(pd.DataFrame(poplist[2]),ignore_index=True)
dfpop = dfpop.append(pd.DataFrame(poplist[3]),ignore_index=True)
dfpop = dfpop.append(pd.DataFrame(poplist[4]),ignore_index=True)
dfpop.columns=['Place']
dfpop

,Place
0,Tarneit
1,PointCook
2,Truganina
3,HoppersCrossing
4,WyndhamVale
5,HamptonPark
6,CranbourneEast
7,Berwick
8,NarreWarrenSouth
9,CranbourneNorth


In [38]:
dfpop=(dfpop.replace("\(Vic.\)",'',regex=True).astype(str))
dfpop['Place'].str.strip()
melb_merged['Place'].str.strip()
melb_merged=melb_merged.reset_index()
melb_merged
#dfpop

,level_0,index,Place,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,0,Melbourne,3001,-37.8140,144.963300,1.0,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
1,1,1,East Melbourne,3002,-37.8133,144.983900,1.0,Café,Australian Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Indian Restaurant,Italian Restaurant,Fish & Chips Shop,Falafel Restaurant,Diner
2,2,2,West Melbourne,3003,-37.8088,144.947300,1.0,Café,Pizza Place,Sandwich Place,Italian Restaurant,Food Truck,Bakery,Noodle House,Mexican Restaurant,Bistro,Himalayan Restaurant
3,3,3,Melbourne,3004,-37.8140,144.967600,1.0,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Korean Restaurant,Asian Restaurant,Burger Joint,Ramen Restaurant,Dumpling Restaurant,Indonesian Restaurant
4,4,4,World Trade Centre,3005,-37.8152,144.963200,1.0,Café,Italian Restaurant,Korean Restaurant,Mexican Restaurant,Sushi Restaurant,Japanese Restaurant,Sandwich Place,Dumpling Restaurant,Burger Joint,Bakery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,80,Diamond Creek,3089,-37.6728,145.153700,1.0,Café,Fish & Chips Shop,Malay Restaurant,Fast Food Restaurant,Yunnan Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
62,62,82,Yarrambat,3091,-37.6390,145.133400,0.0,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant
63,63,84,Montmorency,3094,-37.7188,145.124500,1.0,Café,Chinese Restaurant,Thai Restaurant,Burger Joint,Fast Food Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
64,64,85,Eltham,3095,-37.7043,145.158267,1.0,Café,Pizza Place,Australian Restaurant,Yunnan Restaurant,Fish & Chips Shop,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant


In [39]:
mergedStuff = pd.merge(dfpop,melb_merged, left_on='Place',right_on='Place', how='inner')
mergedStuff


,Place,level_0,index,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Epping,53,69,3076,-37.6307,145.0171,1.0,Italian Restaurant,Café,Yunnan Restaurant,Fish & Chips Shop,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant
1,Lalor,52,68,3075,-37.6673,145.0145,2.0,Bakery,Food,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Egyptian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop
2,Thomastown,51,67,3074,-37.6821,145.0126,1.0,Fish & Chips Shop,Fried Chicken Joint,Café,Middle Eastern Restaurant,Halal Restaurant,Gastropub,French Restaurant,Food Truck,Food Court,Food
